# Parameter Distribution Plot

In [1]:
import sys
sys.path.append('../../../')

In [2]:
import pandas as pd
import numpy as np
from numpy.random import uniform
np.random.seed(89345)

In [3]:
from bokeh.plotting import output_notebook
output_notebook()

Loading BokehJS ...

In [4]:
from estimagic.optimization.utilities import index_element_to_string
from estimagic.visualization.distribution_plot.parameter_distribution_plot import (
    parameter_distribution_plot,
    _tidy_df_from_results
)

## Generate some artifical results

We start by creating some fake results. 
Let's start with a rather minimal version of results 
that only contains the columns that are guaranteed 
to be in the final params DataFrame.

In [5]:
base_params = pd.DataFrame()
n_work_coeffs = 2
n_home_coeffs = 4
n_educ_coeffs = 4
n_beta = n_work_coeffs + n_home_coeffs + n_educ_coeffs
# generate an index
base_params["vartype"] = ["coefficient"] * n_beta + ["cutoff"] * 3
base_params["choice"] = \
    ["work"] * n_work_coeffs + ["home"] * n_home_coeffs + ["educ"] * n_educ_coeffs + \
    ["home", "educ", "work"]
base_params["varname"] = \
    ["x{}".format(i) for i in range(n_work_coeffs)] + \
    ["x{}".format(i) for i in range(n_home_coeffs - 2)] + ["z1", "z2"] + \
    ["x5", "x6", "z3", "z4"] + \
    [None, None, None]

base_params["group"] = "All Parameters"

base_params["value"] = \
    uniform(-3, 3, n_work_coeffs).tolist() + \
    uniform(-1, 3, n_home_coeffs).tolist() + \
    uniform(-3, 1, n_educ_coeffs).tolist() + \
    [0, 1.153, 4.037]


base_params.set_index(["vartype", "choice", "varname"], inplace=True)

base_params["name"] = [index_element_to_string(tup) for tup in base_params.index]
base_params["lower"] = - np.inf
base_params["upper"] = np.inf

In [6]:
base_params

group     value                 name  \
vartype     choice varname                                                  
coefficient work   x0       All Parameters  2.165890  coefficient_work_x0   
                   x1       All Parameters  1.573793  coefficient_work_x1   
            home   x0       All Parameters  2.809119  coefficient_home_x0   
                   x1       All Parameters  0.742688  coefficient_home_x1   
                   z1       All Parameters  1.688022  coefficient_home_z1   
                   z2       All Parameters  2.331855  coefficient_home_z2   
            educ   x5       All Parameters -1.523941  coefficient_educ_x5   
                   x6       All Parameters  0.883510  coefficient_educ_x6   
                   z3       All Parameters -2.359987  coefficient_educ_z3   
                   z4       All Parameters  0.616226  coefficient_educ_z4   
cutoff      home   NaN      All Parameters  0.000000      cutoff_home_nan   
            educ   NaN      All Parameters  1.153000      cutoff_educ_nan   
            work   NaN      All Parameters  4.037000      cutoff_work_nan   

                            lower  upper  
vartype     choice varname                
coefficient work   x0        -inf    inf  
                   x1        -inf    inf  
            home   x0        -inf    inf  
                   x1        -inf    inf  
                   z1        -inf    inf  
                   z2        -inf    inf  
            educ   x5        -inf    inf  
                   x6        -inf    inf  
                   z3        -inf    inf  
                   z4        -inf    inf  
cutoff      home   NaN       -inf    inf  
            educ   NaN       -inf    inf  
            work   NaN       -inf    inf

In [7]:
def create_results(params, n_res=50, model_classes=None, add_model_name=False):
    """Create a list of slightly perturbed versions of a base params DataFrame.
    
    Args:
        params (pd.DataFrame): base version of the params DataFrame.
        n_res (int): number of results to generate.
        model_classes (list): list of model classes
        add_model_name (bool): whether to add a model_name to each DataFrame.
    """
    if model_classes is None:
        model_classes = [None]
    results = []
    for model in model_classes: 
        for i in range(int(n_res / len(model_classes))):
            res = params.copy()
            res["value"] += uniform(-0.75, 0.75, len(res))
            if model is not None:
                res["model_class"] = model
            if add_model_name:
                res["model_name"] = "{}_{}".format(model, i)
            results.append(res)
    return results

In [8]:
minimal_results = create_results(base_params)

## Generate the comparison plot

In [9]:
source, plots = parameter_distribution_plot(
    results=minimal_results,
    height=1000,
)

## Adding model_class and group_col

This is very large and not well sorted. We can use the group_col to kick out groups of parameters we're not interested in. 
Assume for the moment that we are only interested in the wage and education coefficients.

In [10]:
base_params["group"] = \
    ["Wage Coefficients"] * n_work_coeffs + \
    [None] * (n_home_coeffs) + \
    ["Education Coefficients"] * n_educ_coeffs + \
    [None] * 3

Furthermore, we might have different model classes we want to compare.

We might try different specifications, different optimization algorithms or different estimators.

Estimagic allows you to color code parameters from models of the same class in the same color. 

Assume we estimated our model with GMM and maximum likelihood with different starting values. The results will show us how sensitive our results are to the additional assumptions we usually need to make when using maximum likelihood.

In [11]:
mixed_results = create_results(
    params=base_params, 
    n_res=100, 
    model_classes=["GMM", "ML"], 
    add_model_name=True
)

In [12]:
source, plots = parameter_distribution_plot(
    results=mixed_results,
    height=700,
)

Alternatively, we can use the MultiIndex to group our plots as any list of columns or index levels is supported. Note that in this case parameters that don't have an entry in every level would not be plotted.

In [13]:
# fill the index of the cut offs so that they will be plotted
index_filled_results = []
for df in mixed_results:
    new_df = df.reset_index()
    new_df["varname"].fillna("Cut Off", inplace=True)
    index_filled_results.append(new_df)


In [14]:
source, plots = parameter_distribution_plot(
    results=index_filled_results,
    group_cols=["vartype", "choice", "varname"],
    height=1400,
)